Weekday onehot encoding

In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv("../../wy_private/walmart-triptype-classification/dataset/train.csv")
original_train = train.copy()
test = pd.read_csv("../../wy_private/walmart-triptype-classification/dataset/test.csv")
original_test = test.copy()

# Train

In [4]:
train = original_train.copy()

In [5]:
# 반품 발생은 0으로 표기하는 컬럼 추가
# train["ScanCount"] = train["ScanCount"]
# train.loc[train.ScanCount < 0, "ScanCount"] = 0

# Fineline 없는 것은 9999로 명시
train["FinelineNumber"].fillna(value=9999, inplace=True)

# Upc 없는 것은 -1로 명시
train["Upc"].fillna(value=-1, inplace=True)

In [6]:
wd = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3, 
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}

train["Weekday"] = train["Weekday"].apply(lambda x: wd[x])

In [7]:
train = train[train['VisitNumber'] != 2154].reset_index(drop=True)

In [8]:
# VisitNumber, DepartmentDescription으로 groupby. ScanCount sum(총 판매량) 
df = train.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].sum()

df1 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].min()
df1.rename(columns={"ScanCount": "Min"}, inplace=True)

df2 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].max()
df2.rename(columns={'ScanCount': "Max"}, inplace=True)

df3 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].mean()
df3.rename(columns={"ScanCount": "Mean"}, inplace=True)

train = train.merge(df1, how='left', on=["VisitNumber"], copy=True)
train = train.merge(df2, how='left', on=["VisitNumber"], copy=True)
train = train.merge(df3, how='left', on=["VisitNumber"], copy=True)  

train["Range"] = train["Max"] - train["Min"]

In [9]:
df = train[train["DepartmentDescription"].isnull()]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Depts"}, inplace=True)
train = train.merge(df, how='left', on=["VisitNumber"], copy=True) 
train["Null_Depts"].fillna(value=1, inplace=True)
train.loc[train.Null_Depts > 0, "Null_Depts"] = 0

In [10]:
# df = train[train["ScanCount"] < 0]
# df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].ScanCount()
# df.rename(columns={"ScanCount": "ReturnScanCount"}, inplace=True) 
# train = train.merge(df, how='left', on=["VisitNumber"], copy=True)
# train["ReturnScanCount"].fillna(value=1, inplace=True)
# train.loc[train.ReturnScanCount > 0, "ReturnScanCount"] = 0

In [11]:
# Indicator for missing FinelineNumber

df = train[train["FinelineNumber"]==9999]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Fineline"}, inplace=True)
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)
train["Null_Fineline"].fillna(value=1, inplace=True)
train.loc[train.Null_Fineline > 0, "Null_Fineline"] = 0

In [12]:
# 각 VisitNumber 당 총 구매한 FinelineNumber의 수

df = train.groupby(["VisitNumber",  "FinelineNumber"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Fineline"}, inplace=True) 
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)

In [13]:
# 각 소비자 당 총 구매한 UPC의 수

df = train.groupby(["VisitNumber",  "Upc"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Upc"}, inplace=True) 
train = train.merge(df, how='left', on=["VisitNumber"], copy=True) 

In [14]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc
0,999,5,4,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1,1
1,30,7,4,6.053882e+10,1,SHOES,8931.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2
2,30,7,4,7.410811e+09,1,PERSONAL CARE,4504.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2
3,26,8,4,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21
4,26,8,4,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21


In [15]:
# 각 소비자 당 총 구매한 DepartmentDescription의 수

df = train.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].count()

In [16]:
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()

In [17]:
df.rename(columns={"ScanCount": "N_Depts"}, inplace=True) 

In [18]:
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)

In [19]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc,N_Depts
0,999,5,4,6.811315e+10,-1,FINANCIAL SERVICES,1000.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1,1,1.0
1,30,7,4,6.053882e+10,1,SHOES,8931.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2,2.0
2,30,7,4,7.410811e+09,1,PERSONAL CARE,4504.0,1.0,1.0,1.0,0.0,0.0,0.0,2,2,2.0
3,26,8,4,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21,6.0
4,26,8,4,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0,1.0,18.0,4.5,17.0,0.0,0.0,17,21,6.0


In [20]:
df = train.groupby(["VisitNumber"], as_index=False)["ScanCount"].sum()
df.rename(columns={"ScanCount": "Sum"}, inplace=True)
train = train.merge(df, how='left', on=["VisitNumber"], copy=True)

In [21]:
train["Ratio_F_D"] = train["N_Fineline"] / train["N_Depts"]
train["Ratio_U_D"] = train["N_Upc"] / train["N_Depts"]
train.shape

(647046, 19)

In [22]:
train["mean_to_min"] = train["Mean"] / train["Min"]
train.loc[train.mean_to_min==0, "mean_to_min"] = 1
train['max_to_mean'] = train["Max"] / train["Mean"]
train.loc[train.max_to_mean==0, "max_to_mean"] = 1

In [23]:
train.drop("Upc", axis=1, inplace=True)
train.shape

(647046, 20)

In [24]:
# 최빈값 함수
from collections import Counter
def mode(x):
    counts = Counter(x)
    max_count = max(counts.values())
    ls = [x_i for x_i, count in counts.items() if count == count]
    return ls[0]

In [25]:
to_grouped = train[["VisitNumber", "FinelineNumber"]]
mf_fineline = to_grouped.groupby("VisitNumber", as_index=False).agg(mode)

In [26]:
depts_dummies = pd.get_dummies(train.DepartmentDescription)
depts_dummies.drop("HEALTH AND BEAUTY AIDS", axis=1, inplace=True)

In [27]:
depts_dummies = pd.concat([train[["VisitNumber"]], depts_dummies], axis=1)
depts_dummies_grouped = depts_dummies.groupby("VisitNumber", as_index=False).sum()
depts_dummies_grouped.shape

(95673, 68)

In [28]:
def smoothing_depts(data):
    tmp = data[["VisitNumber", "Sum"]]
    tmp = tmp.groupby("VisitNumber", as_index=False).max()
    depts_dummies = pd.get_dummies(data["DepartmentDescription"])
    new_df = pd.concat([tmp, depts_dummies], axis=1)
    new_df = new_df.groupby("VisitNumber", as_index=False).sum()
    new_df = new_df.loc[:, "1-HR PHOTO":"WIRELESS"].apply(lambda x: (x+1) / (tmp["Sum"]+68))
    return new_df.drop(["HEALTH AND BEAUTY AIDS"], axis=1)

In [29]:
depts_smoothed = smoothing_depts(train)

In [30]:
train = train.replace(np.inf, 0)
train = train.fillna(value=0)

In [31]:
train.drop(["DepartmentDescription", "FinelineNumber"], axis=1, inplace=True)

In [32]:
train_by_max = train.groupby("VisitNumber", as_index=False).agg(np.max) # ScanCount --> Sum
train_by_sum = train.groupby("VisitNumber", as_index=False).agg(np.sum) 

In [33]:
train_by_sum.head(10)

,VisitNumber,TripType,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc,N_Depts,Sum,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
0,5,999,4,-1,-1.0,-1.0,-1.000000,0.0,0.0,0.0,1,1,1.0,-1,1.000000,1.000000,1.000000,1.0
1,7,60,8,2,2.0,2.0,2.000000,0.0,0.0,0.0,4,4,4.0,4,2.000000,2.000000,2.000000,2.0
2,8,598,92,28,23.0,414.0,103.500000,391.0,0.0,0.0,391,483,138.0,644,65.166667,80.500000,103.500000,92.0
3,9,24,12,3,3.0,6.0,4.500000,3.0,0.0,0.0,9,9,6.0,9,4.500000,4.500000,4.500000,4.0
4,10,24,12,3,3.0,6.0,4.500000,3.0,0.0,0.0,9,9,6.0,9,4.500000,4.500000,4.500000,4.0
5,11,140,16,4,4.0,8.0,5.333333,4.0,0.0,0.0,16,16,12.0,16,5.333333,5.333333,5.333333,6.0
6,12,287,28,7,7.0,14.0,12.250000,7.0,0.0,0.0,49,49,28.0,49,12.250000,12.250000,12.250000,8.0
7,15,168,32,9,72.0,72.0,72.000000,0.0,0.0,0.0,48,64,8.0,72,48.000000,64.000000,8.000000,8.0
8,17,24,16,4,8.0,8.0,8.000000,0.0,0.0,0.0,16,16,8.0,16,8.000000,8.000000,4.000000,4.0
9,19,378,36,9,9.0,27.0,13.500000,18.0,0.0,0.0,72,72,54.0,81,12.000000,12.000000,13.500000,18.0


In [34]:
train_grouped = train_by_max.merge(depts_dummies_grouped, how='left', on=["VisitNumber"])
# train_grouped = train_grouped.merge(fineline_dummies_grouped, how='left', on=["VisitNumber"])
train_grouped['ScanCount'] = train_by_sum['ScanCount']
train_grouped.shape

(95673, 85)

In [35]:
depts_tmp = train_grouped.loc[:, "1-HR PHOTO":"WIRELESS"]
depts_tmp = depts_smoothed
depts_tmp.columns = depts_tmp.columns.map(lambda x: str("Prob_") + x)

In [36]:
def grouping_depts(data):
    data["OPTICAL"] = data["OPTICAL - LENSES"] + data["OPTICAL - FRAMES"]
    data["DELI"] = data["SERVICE DELI"] + data["PRE PACKED DELI"]

    data["WOMENS CLOTHES"] = data["PLUS AND MATERNITY"] + data["LADIES SOCKS"] +\
                             data["LADIES SOCKS"] + data["BRAS & SHAPEWEAR"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["PLAYERS AND ELECTRONICS"]
    data["LADIESWEAR"] = data["LADIESWEAR"] + data["WOMENS CLOTHES"]

    data["PHARMACY"] = data["PHARMACY RX"] + data["PHARMACY OTC"]
    data["LAWN AND GARDEN"] = data["LAWN AND GARDEN"] + data["HORTICULTURE AND ACCESS"]
    data["LADIESWEAR"] = data["SHEER HOSIERY"] + data["LADIESWEAR"]

    data["CLOTHES"] = data["BOYS WEAR"] + data["MENSWEAR"] + data["SWIMWEAR/OUTERWEAR"] +\
                      data['GIRLS WEAR, 4-6X  AND 7-14'] + data["INFANT APPAREL"] +\
                      data["LADIESWEAR"] + data["MENS WEAR"] + data["SLEEPWEAR/FOUNDATIONS"]

    data["GROCERIES"] = data["BAKERY"] + data["COMM BREAD"] + data["DAIRY"] + data["DSD GROCERY"] +\
                        data["FROZEN FOODS"] + data["GROCERY DRY GOODS"] + data["MEAT - FRESH & FROZEN"] +\
                        data["PRODUCE"] + data["SEAFOOD"] + data["DELI"]

    data["CLOTHES"] = data["CLOTHES"] + data["SHOES"]
    data["HOME"] = data["BATH AND SHOWER"] + data["HOME DECOR"] + data["HOME MANAGEMENT"] +\
                   data["BEDDING"] + data["COOK AND DINE"] + data["FURNITURE"] +\
                   data["LARGE HOUSEHOLD GOODS"] + data["SEASONAL"]

    data["JEWELRY AND ACCESSORIES"] = data["JEWELRY AND SUNGLASSES"] + data["ACCESSORIES"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["CAMERAS AND SUPPLIES"]

    data['HOUSEHOLD GOODS'] = data['HOUSEHOLD PAPER GOODS'] + data['HOUSEHOLD CHEMICALS/SUPP']
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["WIRELESS"]
    data["BEAUTY AND PERSONAL CARE"] = data["BEAUTY"] + data["PERSONAL CARE"]

    to_drop = ["OPTICAL - FRAMES", "OPTICAL - LENSES", "CONCEPT STORES",
               "OTHER DEPARTMENTS", "PRE PACKED DELI", "SERVICE DELI", "PLUS AND MATERNITY", "LADIES SOCKS",
               "BRAS & SHAPEWEAR", "PLAYERS AND ELECTRONICS", "WOMENS CLOTHES", "PHARMACY RX", "PHARMACY OTC",
               "HORTICULTURE AND ACCESS", "SHEER HOSIERY", "BOYS WEAR", "MENSWEAR", "SWIMWEAR/OUTERWEAR",
               "GIRLS WEAR, 4-6X  AND 7-14", "INFANT APPAREL", "LADIESWEAR", "MENS WEAR", "SLEEPWEAR/FOUNDATIONS",
               "BAKERY", "COMM BREAD", "DAIRY", "DSD GROCERY", "FROZEN FOODS", "GROCERY DRY GOODS",
               "MEAT - FRESH & FROZEN","PRODUCE", "SEAFOOD", "DELI", "SHOES", "HOME DECOR", "BATH AND SHOWER",
               "HOME MANAGEMENT", "BEDDING","COOK AND DINE", "FURNITURE", "LARGE HOUSEHOLD GOODS",
               "SEASONAL", "JEWELRY AND SUNGLASSES", "ACCESSORIES", "CAMERAS AND SUPPLIES",
               "HOUSEHOLD CHEMICALS/SUPP", "HOUSEHOLD PAPER GOODS", "WIRELESS", "BEAUTY", "PERSONAL CARE"]
    
    return data.drop(to_drop, axis=1)

In [186]:
train_depts_group = grouping_depts(train_grouped)
train_depts_group = pd.concat([train_depts_group, depts_tmp], axis=1)
train_depts_group.tail(3)

,VisitNumber,TripType,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,...,Prob_SEAFOOD,Prob_SEASONAL,Prob_SERVICE DELI,Prob_SHEER HOSIERY,Prob_SHOES,Prob_SLEEPWEAR/FOUNDATIONS,Prob_SPORTING GOODS,Prob_SWIMWEAR/OUTERWEAR,Prob_TOYS,Prob_WIRELESS
95670,191345,39,6,17,1.0,5.0,2.125,4.0,0.0,0.0,...,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765
95671,191346,39,6,17,1.0,5.0,2.125,4.0,0.0,0.0,...,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765
95672,191347,8,6,2,1.0,1.0,1.000,0.0,0.0,0.0,...,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286


In [187]:
# train_depts_group.drop(9999, axis=1, inplace=True)
train_depts_group = pd.concat([train_depts_group, mf_fineline[["FinelineNumber"]]], axis=1)
# train_depts_group = pd.get_dummies(train_depts_group, columns=["Weekday"])
# train_depts_group.drop(X.iloc[:, 170:357], axis=1, inplace=True)

In [188]:
train_depts_group = pd.get_dummies(train_depts_group, columns=["Weekday"])

In [189]:
train_depts_group['Return_YN'] = train_depts_group['ScanCount'].apply(lambda x : 1 if x < 0 else 0)
train_depts_group['Return_Count'] = train_depts_group['ScanCount'].apply(lambda x: x if x < 0 else 0)
train_depts_group['Return_Count'] = train_depts_group['Return_Count'].apply(abs)

In [190]:
train_depts_group.iloc[:, 44:-10]

,Prob_1-HR PHOTO,Prob_ACCESSORIES,Prob_AUTOMOTIVE,Prob_BAKERY,Prob_BATH AND SHOWER,Prob_BEAUTY,Prob_BEDDING,Prob_BOOKS AND MAGAZINES,Prob_BOYS WEAR,Prob_BRAS & SHAPEWEAR,...,Prob_SEAFOOD,Prob_SEASONAL,Prob_SERVICE DELI,Prob_SHEER HOSIERY,Prob_SHOES,Prob_SLEEPWEAR/FOUNDATIONS,Prob_SPORTING GOODS,Prob_SWIMWEAR/OUTERWEAR,Prob_TOYS,Prob_WIRELESS
0,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,...,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925,0.014925
1,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,...,0.014286,0.014286,0.014286,0.014286,0.028571,0.014286,0.014286,0.014286,0.014286,0.014286
2,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,...,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417
3,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085
4,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085
5,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,...,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889
6,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,...,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333
7,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987
8,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,...,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889
9,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,...,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987


In [61]:
train_depts_group = train_depts_group.drop(columns = train_depts_group.iloc[:, 44:-10])

In [62]:
print(train_depts_group.shape)
train_depts_group.head()

(95673, 54)


,VisitNumber,TripType,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,...,FinelineNumber,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Return_YN,Return_Count
0,5,999,-1,-1.0,-1.0,-1.0,0.0,0.0,0.0,1,...,1000.0,0,0,0,0,1,0,0,1,1
1,7,30,2,1.0,1.0,1.0,0.0,0.0,0.0,2,...,8931.0,0,0,0,0,1,0,0,0,0
2,8,26,28,1.0,18.0,4.5,17.0,0.0,0.0,17,...,3565.0,0,0,0,0,1,0,0,0,0
3,9,8,3,1.0,2.0,1.5,1.0,0.0,0.0,3,...,115.0,0,0,0,0,1,0,0,0,0
4,10,8,3,1.0,2.0,1.5,1.0,0.0,0.0,3,...,2008.0,0,0,0,0,1,0,0,0,0


In [135]:
train = train_depts_group.copy()

# Test

In [64]:
test = original_test.copy()

In [65]:
test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0


In [66]:
# 반품 발생은 0으로 표기하는 컬럼 추가
# test["ScanCount"] = test["ScanCount"]
# test.loc[test.ScanCount < 0, "ScanCount"] = 0

# Fineline 없는 것은 9999로 명시
test["FinelineNumber"].fillna(value=9999, inplace=True)

# Upc 없는 것은 -1로 명시
test["Upc"].fillna(value=-1, inplace=True)

In [67]:
wd = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3, 
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}

test["Weekday"] = test["Weekday"].apply(lambda x: wd[x])

In [68]:
# VisitNumber, DepartmentDescription으로 groupby. ScanCount sum(총 판매량) 
df = test.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].sum()

df1 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].min()
df1.rename(columns={"ScanCount": "Min"}, inplace=True)

df2 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].max()
df2.rename(columns={'ScanCount': "Max"}, inplace=True)

df3 = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].mean()
df3.rename(columns={"ScanCount": "Mean"}, inplace=True)

test = test.merge(df1, how='left', on=["VisitNumber"], copy=True)
test = test.merge(df2, how='left', on=["VisitNumber"], copy=True)
test = test.merge(df3, how='left', on=["VisitNumber"], copy=True)  

In [69]:
test["Range"] = test["Max"] - test["Min"]

In [70]:
df = test[test["DepartmentDescription"].isnull()]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Depts"}, inplace=True)
test = test.merge(df, how='left', on=["VisitNumber"], copy=True) 
test["Null_Depts"].fillna(value=1, inplace=True)
test.loc[test.Null_Depts > 0, "Null_Depts"] = 0

In [71]:
test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts
0,1,4,7.250339e+10,1,SHOES,3002.0,1.0,2.0,1.333333,1.0,0.0
1,1,4,1.707711e+09,1,DAIRY,1526.0,1.0,2.0,1.333333,1.0,0.0
2,1,4,8.947000e+10,1,DAIRY,1431.0,1.0,2.0,1.333333,1.0,0.0
3,1,4,8.849121e+10,1,GROCERY DRY GOODS,3555.0,1.0,2.0,1.333333,1.0,0.0
4,2,4,2.840015e+09,1,DSD GROCERY,4408.0,1.0,2.0,1.333333,1.0,0.0


In [72]:
# df = test[test["ScanCount"] < 0]
# df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].ScanCount()
# df.rename(columns={"ScanCount": "ReturnScanCount"}, inplace=True) 
# test = test.merge(df, how='left', on=["VisitNumber"], copy=True)
# test["ReturnCount"].fillna(value=1, inplace=True)
# test.loc[test.ReturnScanCount > 0, "ReturnScanCount"] = 0

In [73]:
# Indicator for missing FinelineNumber

df = test[test["FinelineNumber"]==9999]
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "Null_Fineline"}, inplace=True)
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)
test["Null_Fineline"].fillna(value=1, inplace=True)
test.loc[test.Null_Fineline > 0, "Null_Fineline"] = 0

In [74]:
# 각 VisitNumber 당 총 구매한 FinelineNumber의 수

df = test.groupby(["VisitNumber",  "FinelineNumber"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Fineline"}, inplace=True) 
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)

In [75]:
# 각 소비자 당 총 구매한 UPC의 수

df = test.groupby(["VisitNumber",  "Upc"], as_index=False)["ScanCount"].count()
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()
df.rename(columns={"ScanCount": "N_Upc"}, inplace=True) 
test = test.merge(df, how='left', on=["VisitNumber"], copy=True) 

train.head()

,VisitNumber,TripType,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,...,FinelineNumber,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Return_YN,Return_Count
0,5,999,-1,-1.0,-1.0,-1.0,0.0,0.0,0.0,1,...,1000.0,0,0,0,0,1,0,0,1,1
1,7,30,2,1.0,1.0,1.0,0.0,0.0,0.0,2,...,8931.0,0,0,0,0,1,0,0,0,0
2,8,26,28,1.0,18.0,4.5,17.0,0.0,0.0,17,...,3565.0,0,0,0,0,1,0,0,0,0
3,9,8,3,1.0,2.0,1.5,1.0,0.0,0.0,3,...,115.0,0,0,0,0,1,0,0,0,0
4,10,8,3,1.0,2.0,1.5,1.0,0.0,0.0,3,...,2008.0,0,0,0,0,1,0,0,0,0


In [76]:
# 각 소비자 당 총 구매한 DepartmentDescription의 수

df = test.groupby(["VisitNumber", "DepartmentDescription"], as_index=False)["ScanCount"].count()

In [77]:
df = df.groupby(["VisitNumber"], as_index=False)["ScanCount"].count()

In [78]:
df.rename(columns={"ScanCount": "N_Depts"}, inplace=True) 

In [79]:
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)

In [80]:
df = test.groupby(["VisitNumber"], as_index=False)["ScanCount"].sum()
df.rename(columns={"ScanCount": "Sum"}, inplace=True)
test = test.merge(df, how='left', on=["VisitNumber"], copy=True)

In [81]:
test["Ratio_F_D"] = train["N_Fineline"] / train["N_Depts"]
test["Ratio_U_D"] = train["N_Upc"] / train["N_Depts"]
test.shape

(653646, 18)

In [82]:
test["mean_to_min"] = test["Mean"] / test["Min"]
test.loc[test.mean_to_min==0, "mean_to_min"] = 1
test['max_to_mean'] = test["Max"] / test["Mean"]
test.loc[test.max_to_mean==0, "max_to_mean"] = 1

In [83]:
test.drop("Upc", axis=1, inplace=True)
test.shape

(653646, 19)

In [84]:
# 최빈값 함수
from collections import Counter
def mode(x):
    counts = Counter(x)
    max_count = max(counts.values())
    ls = [x_i for x_i, count in counts.items() if count == max_count]
    return ls[0]

In [85]:
to_grouped = test[["VisitNumber", "FinelineNumber"]]
mf_fineline = to_grouped.groupby("VisitNumber", as_index=False).agg(mode)

depts_dummies = pd.get_dummies(test.DepartmentDescription)
# depts_dummies.drop("HEALTH AND BEAUTY AIDS", axis=1, inplace=True)

depts_dummies = pd.concat([test[["VisitNumber"]], depts_dummies], axis=1)
depts_dummies_grouped = depts_dummies.groupby("VisitNumber", as_index=False).sum()
depts_dummies_grouped.shape

(95674, 68)

In [86]:
def smoothing_depts(data):
    tmp = data[["VisitNumber", "Sum"]]
    tmp = tmp.groupby("VisitNumber", as_index=False).max()
    depts_dummies = pd.get_dummies(data["DepartmentDescription"])
    new_df = pd.concat([tmp, depts_dummies], axis=1)
    new_df = new_df.groupby("VisitNumber", as_index=False).sum()
#     new_df = new_df.loc[:, "1-HR PHOTO":"WIRELESS"].apply(lambda x: (x+1) / (tmp["Sum"]+68))
#     return new_df.drop(["HEALTH AND BEAUTY AIDS"], axis=1)
    return new_df

depts_smoothed = smoothing_depts(test)

In [87]:
test = test.replace(np.inf, 0)
test = test.fillna(value=0)

In [88]:
test.head()

,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc,N_Depts,Sum,Ratio_F_D,Ratio_U_D,mean_to_min,max_to_mean
0,1,4,1,SHOES,3002.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,1.000000,1.0,1.333333,1.5
1,1,4,1,DAIRY,1526.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,1.000000,1.0,1.333333,1.5
2,1,4,1,DAIRY,1431.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,2.833333,3.5,1.333333,1.5
3,1,4,1,GROCERY DRY GOODS,3555.0,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,3.0,4,1.500000,1.5,1.333333,1.5
4,2,4,1,DSD GROCERY,4408.0,1.0,2.0,1.333333,1.0,0.0,0.0,3,4,3.0,4,1.500000,1.5,1.333333,1.5


In [89]:
test.drop(["DepartmentDescription", "FinelineNumber"], axis=1, inplace=True)

test_by_max = test.groupby("VisitNumber", as_index=False).agg(np.max) # ScanCount --> Sum
test_by_sum = test.groupby("VisitNumber", as_index=False).agg(np.sum) 

In [90]:
test_grouped = test_by_max.merge(depts_dummies_grouped, how='left', on=["VisitNumber"])
# train_grouped = train_grouped.merge(fineline_dummies_grouped, how='left', on=["VisitNumber"])
test_grouped['ScanCount'] = test_by_sum['ScanCount']
test_grouped.shape

(95674, 84)

In [91]:
test_grouped.head()

,VisitNumber,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,1,4,4,1.0,2.0,1.333333,1.0,0.0,0.0,4,...,0,0,0,0,1,0,0,0,0,0
1,2,4,4,1.0,2.0,1.333333,1.0,0.0,0.0,3,...,0,0,0,0,0,0,0,0,0,0
2,3,4,0,0.0,0.0,0.000000,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,4,1,1.0,1.0,1.000000,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
4,6,4,0,0.0,0.0,0.000000,0.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0


In [92]:
depts_tmp_test = test_grouped.loc[:, "1-HR PHOTO":"WIRELESS"]
depts_tmp_test = depts_smoothed
depts_tmp_test.columns = depts_tmp_test.columns.map(lambda x: str("Prob_") + x)

In [93]:
def grouping_depts(data):
    data["OPTICAL"] = data["OPTICAL - LENSES"] + data["OPTICAL - FRAMES"]
    data["DELI"] = data["SERVICE DELI"] + data["PRE PACKED DELI"]

    data["WOMENS CLOTHES"] = data["PLUS AND MATERNITY"] + data["LADIES SOCKS"] +\
                             data["LADIES SOCKS"] + data["BRAS & SHAPEWEAR"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["PLAYERS AND ELECTRONICS"]
    data["LADIESWEAR"] = data["LADIESWEAR"] + data["WOMENS CLOTHES"]

    data["PHARMACY"] = data["PHARMACY RX"] + data["PHARMACY OTC"]
    data["LAWN AND GARDEN"] = data["LAWN AND GARDEN"] + data["HORTICULTURE AND ACCESS"]
    data["LADIESWEAR"] = data["SHEER HOSIERY"] + data["LADIESWEAR"]

    data["CLOTHES"] = data["BOYS WEAR"] + data["MENSWEAR"] + data["SWIMWEAR/OUTERWEAR"] +\
                      data['GIRLS WEAR, 4-6X  AND 7-14'] + data["INFANT APPAREL"] +\
                      data["LADIESWEAR"] + data["MENS WEAR"] + data["SLEEPWEAR/FOUNDATIONS"]

    data["GROCERIES"] = data["BAKERY"] + data["COMM BREAD"] + data["DAIRY"] + data["DSD GROCERY"] +\
                        data["FROZEN FOODS"] + data["GROCERY DRY GOODS"] + data["MEAT - FRESH & FROZEN"] +\
                        data["PRODUCE"] + data["SEAFOOD"] + data["DELI"]

    data["CLOTHES"] = data["CLOTHES"] + data["SHOES"]
    data["HOME"] = data["BATH AND SHOWER"] + data["HOME DECOR"] + data["HOME MANAGEMENT"] +\
                   data["BEDDING"] + data["COOK AND DINE"] + data["FURNITURE"] +\
                   data["LARGE HOUSEHOLD GOODS"] + data["SEASONAL"]

    data["JEWELRY AND ACCESSORIES"] = data["JEWELRY AND SUNGLASSES"] + data["ACCESSORIES"]
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["CAMERAS AND SUPPLIES"]

    data['HOUSEHOLD GOODS'] = data['HOUSEHOLD PAPER GOODS'] + data['HOUSEHOLD CHEMICALS/SUPP']
    data["ELECTRONICS"] = data["ELECTRONICS"] + data["WIRELESS"]
    data["BEAUTY AND PERSONAL CARE"] = data["BEAUTY"] + data["PERSONAL CARE"]

    to_drop = ["OPTICAL - FRAMES", "OPTICAL - LENSES", "CONCEPT STORES",
               "OTHER DEPARTMENTS", "PRE PACKED DELI", "SERVICE DELI", "PLUS AND MATERNITY", "LADIES SOCKS",
               "BRAS & SHAPEWEAR", "PLAYERS AND ELECTRONICS", "WOMENS CLOTHES", "PHARMACY RX", "PHARMACY OTC",
               "HORTICULTURE AND ACCESS", "SHEER HOSIERY", "BOYS WEAR", "MENSWEAR", "SWIMWEAR/OUTERWEAR",
               "GIRLS WEAR, 4-6X  AND 7-14", "INFANT APPAREL", "LADIESWEAR", "MENS WEAR", "SLEEPWEAR/FOUNDATIONS",
               "BAKERY", "COMM BREAD", "DAIRY", "DSD GROCERY", "FROZEN FOODS", "GROCERY DRY GOODS",
               "MEAT - FRESH & FROZEN","PRODUCE", "SEAFOOD", "DELI", "SHOES", "HOME DECOR", "BATH AND SHOWER",
               "HOME MANAGEMENT", "BEDDING","COOK AND DINE", "FURNITURE", "LARGE HOUSEHOLD GOODS",
               "SEASONAL", "JEWELRY AND SUNGLASSES", "ACCESSORIES", "CAMERAS AND SUPPLIES",
               "HOUSEHOLD CHEMICALS/SUPP", "HOUSEHOLD PAPER GOODS", "WIRELESS", "BEAUTY", "PERSONAL CARE"]
    
    return data.drop(to_drop, axis=1)

In [194]:
test_depts_group = grouping_depts(test_grouped)
test_depts_group = pd.concat([test_depts_group, depts_tmp], axis=1)
test_depts_group.tail(3)

,VisitNumber,Weekday,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,...,Prob_SEAFOOD,Prob_SEASONAL,Prob_SERVICE DELI,Prob_SHEER HOSIERY,Prob_SHOES,Prob_SLEEPWEAR/FOUNDATIONS,Prob_SPORTING GOODS,Prob_SWIMWEAR/OUTERWEAR,Prob_TOYS,Prob_WIRELESS
95671,191340,6,2,2.0,2.0,2.000000,0.0,0.0,0.0,1,...,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765,0.011765
95672,191341,6,12,1.0,5.0,2.400000,4.0,0.0,0.0,10,...,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286,0.014286
95673,191348,6,7,2.0,3.0,2.333333,1.0,0.0,0.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
test_depts_group = pd.concat([test_depts_group, mf_fineline[["FinelineNumber"]]], axis=1)
test_depts_group = pd.get_dummies(test_depts_group, columns=["Weekday"])

In [196]:
test_depts_group['Return_YN'] = test_depts_group['ScanCount'].apply(lambda x : 1 if x < 0 else 0)
test_depts_group['Return_Count'] = test_depts_group['ScanCount'].apply(lambda x: x if x < 0 else 0)
test_depts_group['Return_Count'] = test_depts_group['Return_Count'].apply(abs)

In [199]:
test_depts_group = test_depts_group.drop(columns = test_depts_group.iloc[:, 43:-10])

In [200]:
print(test_depts_group.shape)
test_depts_group.head()

(95674, 53)


,VisitNumber,ScanCount,Min,Max,Mean,Range,Null_Depts,Null_Fineline,N_Fineline,N_Upc,...,FinelineNumber,Weekday_0,Weekday_1,Weekday_2,Weekday_3,Weekday_4,Weekday_5,Weekday_6,Return_YN,Return_Count
0,1,4,1.0,2.0,1.333333,1.0,0.0,0.0,4,4,...,3002.0,0,0,0,0,1,0,0,0,0
1,2,4,1.0,2.0,1.333333,1.0,0.0,0.0,3,4,...,8023.0,0,0,0,0,1,0,0,0,0
2,3,0,0.0,0.0,0.000000,0.0,0.0,0.0,1,1,...,4504.0,0,0,0,0,1,0,0,0,0
3,4,1,1.0,1.0,1.000000,0.0,0.0,0.0,1,1,...,5924.0,0,0,0,0,1,0,0,0,0
4,6,0,0.0,0.0,0.000000,0.0,0.0,0.0,1,1,...,654.0,0,0,0,0,1,0,0,0,0


In [201]:
test = test_depts_group.copy()

# Data Split

In [104]:
from sklearn.model_selection import train_test_split

In [105]:
# y = train_depts_group["TripType"]
# X = train_depts_group.drop("TripType", axis=1)

In [202]:
X_train, X_test = train_test_split(train, test_size = .4)

In [203]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(X_train.TripType)
X_train_label = le.transform(X_train.TripType) 

In [204]:
le = LabelEncoder()
le.fit(X_test.TripType)
X_test_label = le.transform(X_test.TripType)

# 1) 왜 XGBoost -- predict로 하면 matrix가 나오는지
# 2) DMatrix를 사용하지 않으면 어떤 다른 방법이 있는지.

# XGBoost Modeling

In [205]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import log_loss

In [206]:
X_train_ = X_train.drop(['TripType'], axis=1)
X_test_ = X_test.drop(['TripType'], axis=1)

In [207]:
dtrain = xgb.DMatrix(np.asarray(X_train_), label = np.asarray(X_train_label, dtype=np.uint32))
dtest = xgb.DMatrix(np.asarray(X_test_), label = np.asarray(X_test_label, dtype=np.uint32))

In [208]:
num_round = 300
param1 = {'objective': 'multi:softprob', 'num_class':38, "max_delta_step": 5}
watchlist1 = [(dtrain,'train'), (dtest, 'eval')]

bst = xgb.train(param1, dtrain, num_round, watchlist1, early_stopping_rounds=10)

[0]	train-merror:0.392837	eval-merror:0.406323
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.333537	eval-merror:0.353697
[2]	train-merror:0.325732	eval-merror:0.348498
[3]	train-merror:0.319896	eval-merror:0.345832
[4]	train-merror:0.315855	eval-merror:0.344343
[5]	train-merror:0.31035	eval-merror:0.340972
[6]	train-merror:0.30542	eval-merror:0.338777
[7]	train-merror:0.30049	eval-merror:0.33653
[8]	train-merror:0.297092	eval-merror:0.335119
[9]	train-merror:0.293556	eval-merror:0.333551
[10]	train-merror:0.290089	eval-merror:0.332584
[11]	train-merror:0.287006	eval-merror:0.331173
[12]	train-merror:0.283348	eval-merror:0.331278
[13]	train-merror:0.280125	eval-merror:0.329579
[14]	train-merror:0.277459	eval-merror:0.329213
[15]	train-merror:0.274463	eval-merror:0.329266
[16]	train-merror:0.270613	eval-merror:0.326757
[17]	train-merror:0.266449	eval-merror:0.326783
[18

In [118]:
X_test_dm = xgb.DMatrix(np.asarray(X_test_))
y_pred = bst.predict(X_test_dm)

In [119]:
y_pred

array([[1.79922022e-03, 1.29603477e-05, 6.56547636e-05, ...,
        7.54306838e-02, 1.28983241e-02, 8.62980261e-03],
       [3.63085273e-05, 3.01383814e-04, 4.78604110e-03, ...,
        7.44352117e-02, 6.42609820e-02, 6.99792756e-04],
       [2.06579298e-05, 3.90368359e-06, 2.34996714e-05, ...,
        5.03179734e-04, 6.58000718e-05, 1.33625707e-02],
       ...,
       [9.02434567e-06, 8.28027027e-04, 1.31955920e-02, ...,
        3.66297178e-03, 4.59754490e-04, 1.01033144e-03],
       [3.32209165e-05, 1.35746359e-05, 7.55024012e-05, ...,
        6.32256779e-05, 6.49029898e-05, 1.94642413e-02],
       [1.09889534e-05, 1.13450915e-05, 1.06010921e-05, ...,
        4.49193067e-05, 1.87257592e-05, 1.09297298e-02]], dtype=float32)

In [120]:
col_array = np.asarray([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37])

In [121]:
columns = {0:"TripType_3", 1: "TripType_4", 2: "TripType_5", 3: "TripType_6", 
                4: "TripType_7", 5: "TripType_8", 6: "TripType_9", 7: "TripType_12", 8: "TripType_14", 
                9: "TripType_15", 10: "TripType_18", 11: "TripType_19", 12: "TripType_20", 13: "TripType_21",
                14: "TripType_22", 15: "TripType_23", 16: "TripType_24", 17: "TripType_25", 18: "TripType_26", 
                19: "TripType_27", 20: "TripType_28", 21: "TripType_29", 22: "TripType_30", 23: "TripType_31", 
                24: "TripType_32", 25: "TripType_33", 26: "TripType_34", 27: "TripType_35", 28: "TripType_36", 
                29: "TripType_37", 30: "TripType_38", 31: "TripType_39", 32: "TripType_40", 33: "TripType_41", 
                34: "TripType_42", 35: "TripType_43", 36: "TripType_44", 37: "TripType_999"}

In [209]:
test_dm = xgb.DMatrix(np.asarray(test))
pred_proba = bst.predict(test_dm)

In [210]:
proba_df = pd.DataFrame(pred_proba, columns=col_array)

In [211]:
proba_df

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.000061,2.161828e-05,4.128426e-05,9.579665e-05,0.105619,0.066250,5.859677e-02,5.704780e-05,8.478024e-06,0.011710,...,3.088687e-04,0.063643,0.026980,0.214005,3.554810e-04,1.081157e-02,1.935923e-02,1.850041e-02,8.184469e-04,0.017401
1,0.000732,6.297540e-05,7.370902e-05,2.380505e-04,0.250648,0.051691,2.739124e-02,2.926266e-05,6.575882e-06,0.083962,...,2.889773e-03,0.027303,0.042507,0.048718,1.599643e-04,4.979763e-04,1.994628e-03,1.061264e-02,5.850021e-04,0.109645
2,0.000013,2.457849e-07,1.238459e-06,1.517551e-06,0.000001,0.000529,4.879125e-05,4.948116e-07,8.765986e-08,0.000006,...,4.326470e-03,0.000003,0.000002,0.000007,1.718502e-06,1.207585e-06,4.645437e-06,4.050793e-07,4.505635e-06,0.995023
3,0.000169,5.896726e-06,2.711178e-05,2.133878e-05,0.000023,0.016008,9.398572e-01,4.425720e-06,5.146820e-05,0.000326,...,5.318185e-05,0.000100,0.000018,0.000057,1.693258e-05,1.532867e-05,3.018893e-04,1.140720e-05,1.116146e-05,0.012384
4,0.000010,4.313148e-07,1.054708e-06,6.850987e-07,0.000004,0.000521,2.399453e-04,5.752682e-07,1.130507e-07,0.000009,...,4.482692e-06,0.000008,0.000001,0.000004,9.112683e-07,2.241087e-05,1.342307e-05,7.773216e-07,1.242090e-06,0.998902
5,0.000353,1.806696e-05,5.967509e-05,6.099536e-05,0.450360,0.518679,3.292597e-03,1.232659e-05,3.737573e-06,0.001103,...,2.419253e-04,0.007541,0.001167,0.000664,8.761604e-05,1.600564e-05,3.104465e-04,4.544581e-05,3.134512e-05,0.012658
6,0.000017,7.946738e-06,2.819277e-05,1.642693e-04,0.112572,0.000264,2.938230e-03,2.297365e-05,5.327134e-06,0.054661,...,5.175876e-04,0.011002,0.072012,0.473506,2.280254e-03,1.113864e-02,4.887018e-02,5.133907e-02,4.730139e-03,0.000645
7,0.000031,3.184194e-05,6.670065e-05,1.766465e-04,0.166860,0.000440,5.757157e-04,5.723138e-05,3.523239e-04,0.064343,...,2.891302e-04,0.019334,0.082965,0.304689,1.399045e-03,9.387225e-04,4.347461e-03,6.815784e-02,5.848617e-04,0.002189
8,0.000041,1.822487e-05,9.157834e-05,1.100357e-04,0.156916,0.000619,2.210387e-03,6.656528e-05,7.652498e-06,0.060792,...,8.601861e-04,0.024235,0.214731,0.467396,6.046263e-04,1.332764e-03,1.491213e-03,1.053848e-02,4.568562e-04,0.002173
9,0.000037,2.367878e-02,6.515909e-01,3.347898e-04,0.000177,0.000210,5.641023e-03,8.976911e-06,2.488086e-06,0.002290,...,1.559502e-01,0.007289,0.000842,0.082518,4.406482e-04,3.458976e-03,1.071344e-02,2.700968e-02,9.756034e-04,0.002253


In [212]:
proba_df.columns = proba_df.columns.map(lambda x: columns[x])

In [213]:
sub_df = pd.concat([test[["VisitNumber"]], proba_df], axis=1)

In [214]:
sub_df

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.000061,2.161828e-05,4.128426e-05,9.579665e-05,0.105619,0.066250,5.859677e-02,5.704780e-05,8.478024e-06,...,3.088687e-04,0.063643,0.026980,0.214005,3.554810e-04,1.081157e-02,1.935923e-02,1.850041e-02,8.184469e-04,0.017401
1,2,0.000732,6.297540e-05,7.370902e-05,2.380505e-04,0.250648,0.051691,2.739124e-02,2.926266e-05,6.575882e-06,...,2.889773e-03,0.027303,0.042507,0.048718,1.599643e-04,4.979763e-04,1.994628e-03,1.061264e-02,5.850021e-04,0.109645
2,3,0.000013,2.457849e-07,1.238459e-06,1.517551e-06,0.000001,0.000529,4.879125e-05,4.948116e-07,8.765986e-08,...,4.326470e-03,0.000003,0.000002,0.000007,1.718502e-06,1.207585e-06,4.645437e-06,4.050793e-07,4.505635e-06,0.995023
3,4,0.000169,5.896726e-06,2.711178e-05,2.133878e-05,0.000023,0.016008,9.398572e-01,4.425720e-06,5.146820e-05,...,5.318185e-05,0.000100,0.000018,0.000057,1.693258e-05,1.532867e-05,3.018893e-04,1.140720e-05,1.116146e-05,0.012384
4,6,0.000010,4.313148e-07,1.054708e-06,6.850987e-07,0.000004,0.000521,2.399453e-04,5.752682e-07,1.130507e-07,...,4.482692e-06,0.000008,0.000001,0.000004,9.112683e-07,2.241087e-05,1.342307e-05,7.773216e-07,1.242090e-06,0.998902
5,13,0.000353,1.806696e-05,5.967509e-05,6.099536e-05,0.450360,0.518679,3.292597e-03,1.232659e-05,3.737573e-06,...,2.419253e-04,0.007541,0.001167,0.000664,8.761604e-05,1.600564e-05,3.104465e-04,4.544581e-05,3.134512e-05,0.012658
6,14,0.000017,7.946738e-06,2.819277e-05,1.642693e-04,0.112572,0.000264,2.938230e-03,2.297365e-05,5.327134e-06,...,5.175876e-04,0.011002,0.072012,0.473506,2.280254e-03,1.113864e-02,4.887018e-02,5.133907e-02,4.730139e-03,0.000645
7,16,0.000031,3.184194e-05,6.670065e-05,1.766465e-04,0.166860,0.000440,5.757157e-04,5.723138e-05,3.523239e-04,...,2.891302e-04,0.019334,0.082965,0.304689,1.399045e-03,9.387225e-04,4.347461e-03,6.815784e-02,5.848617e-04,0.002189
8,18,0.000041,1.822487e-05,9.157834e-05,1.100357e-04,0.156916,0.000619,2.210387e-03,6.656528e-05,7.652498e-06,...,8.601861e-04,0.024235,0.214731,0.467396,6.046263e-04,1.332764e-03,1.491213e-03,1.053848e-02,4.568562e-04,0.002173
9,21,0.000037,2.367878e-02,6.515909e-01,3.347898e-04,0.000177,0.000210,5.641023e-03,8.976911e-06,2.488086e-06,...,1.559502e-01,0.007289,0.000842,0.082518,4.406482e-04,3.458976e-03,1.071344e-02,2.700968e-02,9.756034e-04,0.002253


In [215]:
sub_df.to_csv("submission_xgb_3.csv", index=False)

그 외 적용점 

In [330]:
np.array(X_test.TripType)

array([ 8, 36,  8, ..., 39, 39,  8])

In [240]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import LabelBinarizer

In [245]:
lb = LabelBinarizer()
lb.fit(X_test_label)
y2 = lb.transform(X_test_label)

In [250]:
y2

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
print(classification_report(y2, y_pred))

In [ ]:
dtrain = xgb.DMatrix(data=X, label=np.array([range(len(X))], dtype=np.uint32).T)

def objFunc(..., dtrain):
    indexes=dtrain.get_label().astype('uint32')
    dtrain=X.iloc[indexes]
    return g, h

bst = xgb.train(..., obj=objFunc, ...)

In [331]:
walmart_test = xgb.DMatrix(np.asarray(test))
prediction_xgb = bst.predict(walmart_test)

In [334]:
prediction_xgb.astype(int)
prediction_xgb

array([[2.2835880e-05, 1.0106867e-05, 2.5292809e-05, ..., 2.1066862e-04,
        6.2925195e-05, 8.6761367e-01],
       [5.8068428e-04, 4.2971147e-05, 2.2810197e-04, ..., 3.6148480e-04,
        1.8670391e-04, 1.6760890e-01],
       [3.0411238e-05, 5.4108241e-06, 4.9307309e-05, ..., 3.6149766e-05,
        3.0830887e-04, 2.8928006e-01],
       ...,
       [4.6393918e-03, 1.0913062e-04, 7.6271594e-04, ..., 2.3896897e-03,
        2.1824802e-03, 5.2670218e-02],
       [3.1299838e-05, 5.4110242e-06, 7.9155443e-06, ..., 6.9684570e-04,
        8.3621610e-03, 4.6911049e-03],
       [1.9105099e-05, 7.5711046e-06, 9.7501761e-05, ..., 1.5369616e-03,
        3.1739142e-02, 2.6002913e-03]], dtype=float32)

In [ ]:
def prediction_to_csv(prediction):
    df_VN = pd.DataFrame(columns=["VisitNumber"])
    df_VN["VisitNumber"] = test.index
    df_TT = pd.DataFrame(columns=["TripType"])
    df_TT["TripType"] = prediction
    df_TT = pd.get_dummies(df_TT.TripType).astype(int, cbopy=False)
    df_TT = df_TT.rename(columns = {0:"TripType_3", 1: "TripType_4", 2: "TripType_5", 3: "TripType_6", 
                4: "TripType_7", 5: "TripType_8", 6: "TripType_9", 7: "TripType_12", 8: "TripType_14", 
                9: "TripType_15", 10: "TripType_18", 11: "TripType_19", 12: "TripType_20", 13: "TripType_21",
                14: "TripType_22", 15: "TripType_23", 16: "TripType_24", 17: "TripType_25", 18: "TripType_26", 
                19: "TripType_27", 20: "TripType_28", 21: "TripType_29", 22: "TripType_30", 23: "TripType_31", 
                24: "TripType_32", 25: "TripType_33", 26: "TripType_34", 27: "TripType_35", 28: "TripType_36", 
                29: "TripType_37", 30: "TripType_38", 31: "TripType_39", 32: "TripType_40", 33: "TripType_41", 
                34: "TripType_42", 35: "TripType_43", 36: "TripType_44", 37: "TripType_999"})
    trial = pd.concat([df_VN, df_TT], axis=1).set_index('VisitNumber')
    #trial["TripType_4"] = 0
    trial["TripType_14"] = 0
    return trial.to_csv("data/trial_6.csv")